In [1]:
'''
Skript  zur Statistische Merkmalsberechnung für die Häuserklassifikation nach Hecht (2014)
Author: Nils Weyrauch

Packages:
Geopandas (+Dependencies)
Pandas
Shapely
fiona
rasterio
numpy

Aufbau:
1. Defintion der Funktionen für die Berechnung der Merkmale
2. Integration der Input Daten 
3. Berechnung der Merkmale mit Nebengebäude
4. Berechnung der Merkmale ohne Nebengebäude
5. Export der Daten 


Daten: (zuletzt überprüft: 19.04.2024)
ALKIS Grundrissdaten im NAS Format: https://www.opengeodata.nrw.de/produkte/geobasis/lk/akt/gru_xml/
(anschließende Umwandlung in Shape Dateien)
Gebäudereferenzen im txt Format: https://www.opengeodata.nrw.de/produkte/geobasis/lk/akt/gebref_txt/
(anschließende Umwandlung zu Shape Datei)
Digitale Verwaltungsgrenzen NRW: https://www.opengeodata.nrw.de/produkte/geobasis/vkg/dvg/dvg1/
Normalisiertes Digitales Oberflächenmodell: https://www.opengeodata.nrw.de/produkte/geobasis/hm/ndom50_tiff/ndom50_tiff/
(Abfrage über Geoporatl NRW, für die Landkreise: Wuppertal, Remscheid, Ennepe Ruhr Kreis, Oberbergischer Kreis || ca. 2500 Einzelkacheln)
'''


'\nSkript  zur Statistische Merkmalsberechnung für die Häuserklassifikation nach Hecht (2014)\nAuthor: Nils Weyrauch\n\nPackages:\nGeopandas (+Dependencies)\nPandas\nShapely\nfiona\nrasterio\nnumpy\n\nAufbau:\n1. Defintion der Funktionen für die Berechnung der Merkmale\n2. Integration der Input Daten \n3. Berechnung der Merkmale mit Nebengebäude\n4. Berechnung der Merkmale ohne Nebengebäude\n5. Export der Daten \n\n\nDaten: (zuletzt überprüft: 19.04.2024)\nALKIS Grundrissdaten im NAS Format: https://www.opengeodata.nrw.de/produkte/geobasis/lk/akt/gru_xml/\n(anschließende Umwandlung in Shape Dateien)\nGebäudereferenzen im txt Format: https://www.opengeodata.nrw.de/produkte/geobasis/lk/akt/gebref_txt/\n(anschließende Umwandlung zu Shape Datei)\nDigitale Verwaltungsgrenzen NRW: https://www.opengeodata.nrw.de/produkte/geobasis/vkg/dvg/dvg1/\nNormalisiertes Digitales Oberflächenmodell: https://www.opengeodata.nrw.de/produkte/geobasis/hm/ndom50_tiff/ndom50_tiff/\n(Abfrage über Geoporatl NRW,

In [2]:
from numpy import sqrt, pi
import geopandas as gpd
import shapely
from pathlib import Path
import fiona 
import pandas as pd
import shapely.geometry 
import rasterio 
from rasterio.mask import mask

In [3]:
# Definieren der Funktionen für spätere Berechnung der statistischen Merkmale

In [4]:
"""
Funktion um die Anzahl aller benachbarten Einzelgebäude festzustellen
Merkmal: SUM_AnzPo

Aufbau: 
1. Alle überschneidenden Polygone ("intersection") in Dictionary festhalten
2. Zähler anlegen, welcher für jede gemeinsame Kante ("touch" Operator) um eins nach oben geht
3. Werte zurückgeben an IP GDF
"""

def alt_Sum_AnzPo(gdf, col_name): 

    # Anlegen eines spatial index für schnellere computation
    spatial_index = gdf.sindex  
    
    # Erstellen eines Dictionaries, hier wird die Anzahl der benachbarten Polygone pro index festgehalten
    neighbor_counts = {}

    # Loop über jede Zeile des GDF
    for index, polygon in gdf.iterrows(): 
        
        # Der Spatial index wird auf überlappung getestet,
        # alle überlappenden indices werden festgehalten, zum check ob Kanten sich berühren
        # spart Zeit gegenüber check von allen Polygonen auf gleiche Kanten
        inters_idx = list(spatial_index.intersection(polygon.geometry.bounds))
        
        # Aussortieren des zu überprüfenden Polygons zur Fehlervermeidung
        nearby_indices = [i for i in inters_idx if i != index]
        
        # Variable für Zählen wird festgehalten
        neighbor_count = 0
        
        # Geometrien werden, auf Basis des Indexes aus Liste inters_pol, aus input Geodataframe aufgerufen,
        # für die Geometrien wird auf "touch" Operation gecheckt (= check auf gleiche Kante)
        # falls Ergebis True steigt der counter um 1
        # falls keine überlappenden Polygone gefunden wurden wird die 0 übertragen
        for i in nearby_indices:
            nearby_polygon = gdf.loc[i, 'geometry']
            if polygon.geometry.touches(nearby_polygon):
                neighbor_count += 1
        
        # Das Wertepaar index : Nachbarschaftscount wird in ursprünglichem Dictionary festgehalten
        neighbor_counts[index] = neighbor_count

    # "Nachbarn" Spalte des ursprünglichen Dataframes wird anhand des indexes geupdated
    gdf[col_name] = gdf.index.map(neighbor_counts.get)

    return gdf


In [5]:
"""
Funktion um die Anzahl aller benachbarten Einzelgebäude festzustellen ohne Einbezug der Nebengebäude
Benötigt vorherige Berechnung der Hauskoordinatenanzahl für die Filterung

Merkmal: w_Sum_AnzPo

Aufbau:
selbes Prinzip wie SUM_AnzPo, nur mit vorheriger Filterung der Nebengebäude
Index Reset nötig durch lückenhafte Indizierung für Loop
"""

def w_Sum_AnzPo(gdf, col_name): 

    # Filtern der Nebengebäude
    gdf_sel = gdf[gdf["HK_ANZ"] >= 1]

    #Index wird Resetted da, durch Filterung index Lücken aufweist, führt sonst zu fehler
    gdf_sel = gdf_sel.reset_index(names="orig_id")

    # Anlegen eines spatial index für schnellere computation
    spatial_index = gdf_sel.sindex  
    
    # Erstellen eines Dictionaries, hier wird die Anzahl der benachbarten Polygone pro index festgehalten
    neighbor_counts = {}

    # Loop über jede Zeile des GDF
    for index, polygon in gdf_sel.iterrows(): 
        
        # Der Spatial index wird auf überlappung getestet,
        # alle überlappenden indices werden festgehalten, zum check ob Kanten sich berühren
        # spart Zeit gegenüber direktem check auf gleiche Kanten
        inters_idx = list(spatial_index.intersection(polygon.geometry.bounds))
        
        # Aussortieren des zu überprüfenden Polygons zur Fehlervermeidung
        nearby_indices = [i for i in inters_idx if i != index]
        
        # Variable für Zählen wird festgehalten, Grundlage für counter
        neighbor_count = 0
        
        # Geometrien werden, auf Basis des Indexes aus Liste inters_pol, aus input Geodataframe aufgerufen,
        # für die abgerufenen Indeces, wird auf "touch" Operation gecheckt
        # falls Ergebis True steigt der counter um 1
        # falls keine überlappenden Polygone gefunden wurden wird die 0 übertragen
        for i in nearby_indices:
            nearby_polygon = gdf_sel.loc[i, 'geometry']
            if polygon.geometry.touches(nearby_polygon):
                neighbor_count += 1
        
        # Das Wertepaar index : Nachbarschaftscount wird in ursprünglichem Dictionary festgehalten
        neighbor_counts[index] = neighbor_count

    # "Nachbarn" Spalte des selektierten Dataframes wird anhand des indexes geupdated
    gdf_sel[col_name] = gdf_sel.index.map(neighbor_counts.get)

    # um die Werte auf den Input GDF zurückzumergen werden orig_id und Werte zurückgemergt auf die orig_id
    gdf = gdf.merge(gdf_sel[["orig_id", col_name]], left_index=True, right_on="orig_id", how="left")

    #droppen der Hilfsspalte
    gdf.drop(columns=["orig_id"], inplace=True)
   
    return gdf

In [6]:
"""
Funktion welche die Größe der Polygone kalkuliert

Merkmal: Area, Length
"""

def calc_area_length(gdf): 

    #Errechnet die Größe und den Umfang des Polygons auf Basis der Geometry Spalte und fügt sie diesem wieder hinzu
    gdf["area"] = gdf["geometry"].area 
    gdf["length"] = gdf["geometry"].length
    
    return gdf
    

In [7]:
"""
Funktion um die Einzelgebäude zu Regionen zusammenfassen, 
dissagregiert die einzelnen Geometrien zusammen von allen baulich zusammenhängenden Gebäuden

Aufbau:
1. Auflösen der Gebäudegeometrien von allen baulich zusammenhängenden Gebäuden (vgl. "Dissolve"-Werkzeug GIS)
2. Berechnung der Größe der neuen Polygone (Gebäuderegionen)
3. Berechnung der Richardson Compactness für die Regionen
3. Rückgabe der Gebäuderegionen, herausfiltern der unbrauchbaren Daten
"""

def create_Geb_Reg(gdf): 

    # unary union fügt alle Polygone zu einem zusammen
    merged_polygons = gdf.geometry.unary_union
    geb_reg = gpd.GeoDataFrame(geometry=[merged_polygons], crs="EPSG:25832")
    
    # explode wandelt alle mulitpart Polygone zu single part Polygonen um
    geb_reg = geb_reg.explode(index_parts=True)

    #Berechne die Größe der Gebäuderegion und hänge diese als "SUM_Area" an
    geb_reg["SUM_AREA"] = geb_reg.area
    geb_reg["Reg_rich_comp"] = (2 * sqrt(geb_reg.area * pi))/geb_reg.length
    
    #Filtern der nutzbaren Spalten, da sonst alle IP-Polygon Spalten übernommen werden
    # welche nur noch unbrauchbare Daten beinhalten
    geb_reg_filt = geb_reg[["geometry", "SUM_AREA", "Reg_rich_comp"]]
    
    return geb_reg_filt
    

In [8]:
"""
Funktion um die Statistischen Merkmale der bewohnten Häuser innerhalb der Gebäuderegion zu berechnen

Merkmale: w_MIN_AREA, w_MAX_AREA
Aufbau:
1. Index resetten und festhalten für Rückgabe ("merge") der Werte auf ursprünglichen Datensatz
2. Filtern der Nebengebäude
3. Häuser auf ihre zugehörige Gebäuderegion joinen
4. Gruppieren des Datensatzes pro Gebäuderegion
5. kleinste und größte Fläche pro Gebäuderegion berechnen
6. zurück-"mergen" auf ungefilterten Datensatz, festhalten der zugehörigen Gebäuderegion ID
"""

def geb_reg_stat(gdf, geb_reg):
    
    # Index wird resettet für spätere Merges, neue Variable muss festgelegt werden sonst entstehen Errors
    gdf_res = gdf.reset_index(drop=False).rename(columns={'index': 'temp_idx'})
    
    #filtern der Nebengebäude für die w_ Statistiken
    gdf_filt = gdf_res[gdf_res["HK_ANZ"] >= 1]

    # Sjoin der Gebäuderegionen auf die Gebäude um Id der Regionen zu erhalten für Groupen
    # Gruppieren der Häuser nach ihrer jeweiligen Region
    gdf_sjoin = gpd.sjoin(gdf_filt, geb_reg, how="left", predicate="intersects")
    
    #Gruppieren des Datensatzes nach ID der Gebäuderegion, welche automatisch als Feld "index_right1" angelegt wird
    gdf_group = gdf_sjoin.groupby("index_right1")
    
    
    # Ermitteln der größten und kleinsten Gebäudegrößen der jeweiligen Gebäuderegion pro "index_right1"
    # Ergenbiss sind PD Series, welche die min und max Werte pro Gebäuderegion enthalten
    max_area = gdf_group["area"].max()
    min_area = gdf_group["area"].min()
    
    # Umbenennen der Series zu den Spaltennamen für Ergebnis
    MAX = max_area.rename("w_MAX_AREA")
    MIN = min_area.rename("w_MIN_AREA")
    
    
    #Zurück mergen der ermittelten Ergebnisse auf den Sjoin per "index_right1" und des jew. index
    gdf_sjoin = gdf_sjoin.merge(MAX, left_on="index_right1", right_index=True)
    gdf_sjoin = gdf_sjoin.merge(MIN, left_on="index_right1", right_index=True)


    #Zurückmergen der Ergebnisse auf den ursprünglichen Datensatz mittels oben erstellten orig_idx Spalte, inplace sorgt dafür dass Datensatz nicht kopiert wird, sondern der ursprüngliche Verändert wird
    #Id der Gebäuderregion wird festgehalten als Spalte für weitere Operationen
    gdf_res = gdf_res.merge(gdf_sjoin[["temp_idx","index_right1", "w_MAX_AREA", "w_MIN_AREA", "SUM_AREA", "Reg_rich_comp"]], on="temp_idx", how="left")
    gdf_res.drop(columns=["temp_idx"], inplace=True)
    gdf_res.rename(columns={"index_right1" : "Reg_ID"}, inplace=True)

    return gdf_res
        

In [9]:
"""
Funktion zum Reklassifiezieren der ALKIS NRW "Gebäudefunktionswerte" in 3 Nutzungsklassen 
1=Wohnfunktion, 2=Wirtschafts und Industriegebäude, 3 = öffentliches Gebäude
alle Gebäude mit partieller Wohnnutzung werden als Wohngebäude betrachtet

Merkmal: Recl_Nutz

Aufbau:
1. Anlegen der neuen Spalte,
2. .loc Funktion festlegen, welche die Werte entsprechend des Felds "gebaeudefu" reklassifiziert
3. check für nicht reklassifizierte Werte
"""

def Reclass_GebFunkt(gdf):

    # Anlegen der Reklassifizierten Nutzungsspalte
    gdf["Recl_Nutz"] = 0

    # .loc Funktion greift auf Werte auf Basis der spezifizierten konditionen zu
    # und weist die entsprechenden neuen Werte zu

    gdf.loc[(gdf["gebaeudefu"] <= 1312) | (gdf["gebaeudefu"] == 2310) | (gdf["gebaeudefu"] == 2320), "Recl_Nutz"] = 1
    
    gdf.loc[(gdf["gebaeudefu"] >= 2000) & ((gdf["gebaeudefu"] <= 2220) | ((gdf["gebaeudefu"] >= 2400) & (gdf["gebaeudefu"] <= 2740)) & (gdf["gebaeudefu"] != 2463)), "Recl_Nutz"] = 2
  
    gdf.loc[(gdf["gebaeudefu"] >= 3000) & (gdf["gebaeudefu"] <= 3290), "Recl_Nutz"] = 3

    #Neue Spalte zählen, um unklassifizierte Werte festzustellen
    print(gdf["Recl_Nutz"].value_counts())

    return gdf

In [10]:
"""
Funktion ermittelt die Anzahl von Punkten innerhalb eines Polygons

Merkmal: HK_Anz

Aufbau:
1. Adressen auf zugehörige Häuser joinen
2. GDF pro Gebäude gruppieren
3. Gruppengröße zählen
4. Rückgabe als Feld "HK_ANZ"
5. n.a Werte (weil keine Adresse vorhanden ist) mit 0 ersetzen
"""

def count_points_in_polygons(points, gdf, polygon_id="gml_id"):

    # Zählen der Punkte in einem Polygon mittels räumlicher Verknüpfung, 
    # "inner" sorgt dafür, dass nur gezählt wird, falls Hausnummer vorliegt, 
    # "groupby" gibt Attribut vor, für welches gezählt wird, hier: Häuseridentifier
    # "size" gibt die Anzahl der Punkte pro gml_id an, neue Spalte wird umbenannt
    HK_pro_Haus = (gdf.sjoin(points, how="inner").groupby(polygon_id).size().rename("HK_ANZ"))

    # Join der Hausnummern Anzahl auf die Häuser, für deren identifier "gml_id"
    # "left" lässt die Geometrien der Punkte zurück, NoData/NAN werden mit 0 gefüllt
    result_gdf = (gdf.join(HK_pro_Haus,on=polygon_id, how="left").fillna({"HK_ANZ": 0}))

    return result_gdf

In [11]:
"""
die Funktion berechnet die Standartabweichungen der Gebäudegrößen pro Gebäudeblock (target_pol) ohne Nebengebäude

Merkmal: w_Std_Area

Aufbau:
1. Filtern der Nebengebäude
2. Häuser pro Gebäudeblock gruppieren
3. Standartabweichung der Gebäudegrößen berechnen
4. Daten zurückgeben an IP GDF
"""

def w_STD_AREA(ip_gdf): #falls nötig (ip_gdf, target_pol):

    #ip_gdf muss auf Nebengebäude gefiltert werden, da w_merkmale ohne Nebengebäude sind
    ip_gdf_filter = ip_gdf[ip_gdf["HK_ANZ"] >= 1]
    
    # area Spalte des IP schon vrohanden, deshalb gruppieren pro Block und die Std der Größe ermitteln,
    # um Ergebnisse nicht zu verfälschen durch verschnittene Häuser wird die ganze Größe eines Hauses in einem Block betrachtet
    grouped = ip_gdf_filter.groupby("block_id")
    std_areas = grouped["area"].std()
    
    # Dataframe wird mit ursprünglichem IP gemergt um Werte zurück auf Input zu spiegeln
    # gemergt wird per ermittelter block_id, auf den Index des Dataframes
    ip_gdf = ip_gdf.merge(std_areas.rename("w_STD_AREA"), left_on="block_id", right_on="block_id", how="left")

    return ip_gdf

In [12]:
"""
Funktion zur Berechnung des Flächenanteils aller Polygone (Häuser) in einem größeren Polygon (Baublock)
Flächenanteil wird auf alle Häuser bezogen zurückgegeben und nicht einzeln

Merkmal: BLABERAT

Aufbau:
1. Größe des Gebäudeblocks berechnen
2. Häuser auf zugehörige Blocks joinen
3. Gruppieren pro Block Index
4. Fläche der Häuser in Gebäudeblock berechnen
5. durch die Fläche des Blocks teilen
6. Anteil zurückgeben an Häuserdatensatz
"""

def summarize_polygon_area_within_polygons(polygons, target_polygons):

    # Calculate the area of each polygon in the target_polygons GeoDataFrame
    target_polygons["area_block"] = target_polygons.area

    #Spatial join, der die Häuser zu den Blocks matched. um alle Häuser zu erhalten wird left festgelegt. 
    # Aufgrund von Häusern die in mehreren blocks liegen muss "intersect" gewählt werden, sodass die Polygone in beide gematcht werden
    overlapping_polygons = gpd.sjoin(polygons, target_polygons, how="left", predicate="intersects")

    # Polygone des Spatial joins werden nach Block gruppiert
    grouped_polygons = overlapping_polygons.groupby("index_right")

    # Aufgrund der Polygon, die in mehreren Blocks vorliegen muss die überschneidende Area neuberechnet werden, 
    # um die korrekten Anteile der überlappenden Parts festzustellen, Sum Areas ethält die intersecting Area der Polygone pro Baublock ID
    sum_areas = grouped_polygons["geometry"].apply(lambda x: x.unary_union.area)

    # um bei der Proportionen berechnung die Indices eins zu eins zu matchen werden die Größen der Blocks gruppiert nach ihrem Index 
    area_blocks = target_polygons.groupby(target_polygons.index)["area_block"]
    
    # Berechnung der Proportionen der Intersection Fläche an der Blockfläche
    proportions = sum_areas / grouped_polygons["area_block"].first()

    # Die Proportionen Series wird in einen Dataframe umgewandelt um sie später zruückzu mergen
    proportions_df = pd.DataFrame(proportions, columns=["BLABERAT"])

    # Reseten des Index um dieses als Spalte zu bekommen, Inplace erlaubt, dass dies im gleichen Datensatz vorgeht
    proportions_df.reset_index(inplace=True)

    # Proportionen werden auf die Sjoin Polygone zurückgespiegelt, da dort der Index vorhanden ist
    overlapping_polygons = overlapping_polygons.merge(proportions_df, on="index_right", how="left")

    return overlapping_polygons

In [13]:
"""
Kalkuliert die Richardson Compactness (Rundheitsmaß) für den Input
Rich_comp = 2 * sqrt(pi * area) / perimeter
"""

def richardson_compactness(gdf):

    gdf["rich_comp"] = (2 * sqrt(gdf["area"] * pi))/gdf["length"]

    return gdf

In [14]:
"""
Funktion kalkuliert das Volumen und Höche eines Polygons, 
indem es die Rasterwerte eines normalisierten DEMs in den Polygongrenzen aufsummiert

Merkmal: vol, height

Aufbau:
1. Anlegen von Listen, welche die Werte speichern
2. NDOM importieren
3. Raster auf Polygone zuschneiden
4. Höhe und Voloumen des Gebäude ermitteln, Werte in Listen speichern
5. Listen als Spalten des Datensatzes speichern
"""

def calc_vol(gdf, ndem): 

    #Erstellen einer leeren Liste, welche die Volumen speichert und einer für die Höhe
    volumes = []
    heights = []
    
    #import NDOM
    with rasterio.open(ndem) as ip_raster:

        #Loop über alle Polygone
        for index, row in gdf.iterrows():

            # extrahieren der geometrien für Maske
            geom = row["geometry"]
            
            # Raster wird per Maske auf geom geclipt (crop=True), 
            # additional OP der funktion wird nicht benötigt deswegen _
            clip_raster, _ = mask(ip_raster, [geom], crop=True)

            # Die geklippten Rasterwerte werden aufsummiert, was dem Volumen des Gebäude entspricht
            # Der durchschnittliche Rasterwert entspricht der Gebäudehöhe
            vol = clip_raster.sum()
            height_val = float(clip_raster.mean())

            #Die Volumen und Höhen werden der Liste angefügt
            volumes.append(vol)
            heights.append(height_val)
            
    
    #Höhen und Volumen werden zurückgegeben an Häuserdatensatz
    gdf.loc[:, "vol"] = volumes
    gdf.loc[:, "height"] = heights
    
    return gdf

In [16]:
#Import der Basisdaten, Filtern der Kreise auf UG per Name

ip_kreise = gpd.read_file(r"F:\Basisdaten\dvg1krs_nw.shp", crs="EPSG:25832")
UG_kreise = ip_kreise[ip_kreise['GN'].isin([ "Ennepe-Ruhr-Kreis",  "Oberbergischer Kreis", "Wuppertal", "Remscheid" ])]
adressen = gpd.read_file(r"F:\Basisdaten\Geb_ref_UG.shp")
ip_Baublock = gpd.read_file(r"F:\Basisdaten\Baublock_UG.shp", crs="EPSG:25832")

#import der ALKIS Daten des UG
alk_wup = gpd.read_file(r"F:\Basisdaten\ALKIS\AX_gebauede.shp", crs="EPSG:25832")
alk_rem = gpd.read_file(r"F:\Basisdaten\ALKIS\ERK_alk.shp", crs="EPSG:25832")
alk_obk = gpd.read_file(r"F:\Basisdaten\ALKIS\OBK_alkis.shp", crs="EPSG:25832")
alk_erk = gpd.read_file(r"F:\Basisdaten\ALKIS\Rem_alkis.shp", crs="EPSG:25832")

#Merge der ALKIS Datasets zu einem einzigen
ALKIS_UG = gpd.GeoDataFrame(pd.concat([alk_wup, alk_rem, alk_obk, alk_erk]), crs="EPSG:25832")

#Reset des indexes um Duplikate zu vermeiden, drop=True alter index wird gelöscht, inplace=True Datensatz wird für default=false kopiert
ALKIS_UG.reset_index(drop=True, inplace=True)

C:\Users\Nils\anaconda3\envs\geopandas\Lib\site-packages\geopandas\array.py:1459: UserWarning: CRS not set for some of the concatenation inputs. Setting output's CRS as ETRS89 / UTM zone 32N (the single non-null crs provided).
  return GeometryArray(data, crs=_get_common_crs(to_concat))


In [17]:
#Droppen aller überflüssigen Daten, die im Datensatz vorhanden sind
important_col = ["gml_id", "gebaeudefu"]
ALKIS_UG_cl = gpd.GeoDataFrame(ALKIS_UG[important_col], geometry=ALKIS_UG.geometry, crs=ALKIS_UG.crs)

In [18]:
# Anwendung aller definierten Funktionen um die Merkmale für die Klassifikation zu generieren
# Zuerst alle Merkmale, die die Nebengebäude miteinbeziehen
# Schritt 1: kalkulieren von Größe und Umfang
alk_1 = calc_area_length(ALKIS_UG_cl)

In [19]:
# Schritt 2: Ermittlung der Anzahl der Hausadressen pro Gebäude, 
# erlaubt später Filtern der Nebengebäude durch Auswahl aller Gebäude mit mindestens einer Adresse
alk_2 = count_points_in_polygons(adressen, alk_1)

In [20]:
# Schritt 3: Ermittlung der Anzahl der Gebäude mit gemeinsamer Kante
alk_3 = alt_Sum_AnzPo(alk_2, "SUM_AnzPO")

In [21]:
# Schritt 4: Berechnung des Blockbezogenen Gebäudeanteils, 
# 7 NAN Werte ,da sie ausserhakb der Baublöcke liegen und keine Intersection aufweisen 
alk_4 = summarize_polygon_area_within_polygons(alk_3, ip_Baublock)

In [22]:
# Renaming von "index_right" zu Block_id für Klarheit, im Datensatz
alk_4.rename(columns={'index_right': 'block_id'}, inplace=True)
# Removen der 7 Gebäude ausserhald von Blockgrenzen
alk_4.dropna(subset=["block_id"], inplace=True)

In [23]:
# Schritt 5: Errechnen der Richardson Compactness
alk_5 = richardson_compactness(alk_4)

In [24]:
# Berechnung aller Merkmale die auf Nebengebäude verzichten

# Schritt 6: Berechnung der Std der Häusergrößen ohne Nebengebäude

alk_6 = w_STD_AREA(alk_5)

# Datensatz enthält NaN Werte für alle Baublöcke welche nur ein Polygon/Haus enthalten

In [25]:
# Schritt 7: Berechnung Anzahl der benachbarten Gebäude mit gemeinsamer Kante ohne Nebengebäude
alk_7 = w_Sum_AnzPo(alk_6, "w_SUM_AnzPo")

In [26]:
# Reset des Index, da dieser durch "w_Sum_AnzPo" NAN und 0 Werte aufweist
alk_7.reset_index(inplace=True, drop=True)

In [27]:
# Erstellen der Gebäuderegionen
geb_reg = create_Geb_Reg(alk_7)

In [28]:
# Schritt 8: Erstellen der Gebäuderegion spezifischen Statistiken auf Basis der Häuserdaten
alk_8 = geb_reg_stat(alk_7, geb_reg)

In [31]:
# Schritt 9: Reklassifizieren der Gebäudefunktion, filtern von ungültigen Werten
alk_9 = Reclass_GebFunkt(alk_8)
alk_10 = alk_9[alk_9["Recl_Nutz"] != 0]

Recl_Nutz
1    270233
2    145335
0     97775
3      6949
Name: count, dtype: int64


In [32]:
# Schritt 10: Errechnen des Volumen und der Höhe für jedes Gebäude,
ndom_ug = r"F:\Basisdaten\ndom_merge.tif"
alk_11 = calc_vol(alk_10, ndom_ug)

C:\Users\Nils\anaconda3\envs\geopandas\Lib\site-packages\geopandas\geodataframe.py:1543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
C:\Users\Nils\anaconda3\envs\geopandas\Lib\site-packages\geopandas\geodataframe.py:1543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [33]:
#Schritt 11: Fallen lassen aller Zeilen, welche unvollständige Daten aufweisen

# Liste festlegen mit collumns die keine NaN Werte haben dürfen für Klassifikation, 
clean_col = ["w_STD_AREA", "BLABERAT", "rich_comp"]
# Fallen lassen aller NaN Werte, da Random Forest nicht mit NoData umgehen kann
Merkmale_clean = alk_11.dropna(subset=clean_col)

In [34]:
# Festlegen der Columns, welche unverwendbare Daten enthält
columns_drop = ["ART", "GN", "KN", "STAND", "FID_gem_UG", "oid_1", "aktualit", "art_1", "name", "schluessel", "gemeinde", "gmdschl", "stadtteil", "block", "gid", "objectid_1", "refname", "FID_line_t", "FID_OBK_ER"]

In [35]:
# Aufräumen des Datensatzes, Spalten aus festgelegter Liste droppen
Merkmale_clean.drop(columns=columns_drop, inplace=True)

C:\Users\Nils\AppData\Local\Temp\ipykernel_2940\1716140455.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Merkmale_clean.drop(columns=columns_drop, inplace=True)


In [36]:
# Export des Datensatzes für Weiterverwendung in ARCGIS
Merkmale_clean.to_file(r"F:\Zwischendaten\Data_Hausmerkmale.shp")

C:\Users\Nils\AppData\Local\Temp\ipykernel_2940\979086632.py:2: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  Merkmale_clean.to_file(r"F:\Zwischendaten\Data_Hausmerkmale.shp")
